In [29]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
from sklearn.preprocessing import StandardScaler
data = pd.read_csv('final_merged_data.csv')
print(data.columns)

Index(['Company Name_x', 'TICKER', 'Quarter', 'Price', 'Total Ret',
       'Price Ret Ex-Dividend', 'Weighted Mkt Return', 'Volume', 'Dividend',
       'Price Low', 'Price High', 'Shares Outstanding', 'Excess Return',
       'Mkt Cap', 'Price Growth Rate', 'Quart Rev', 'Quart NI', 'Equity',
       'Liabilities', 'Revenue', 'Industry Code', 'Asset', 'Current Asset',
       'EPS', 'Rating', 'Revenue Growth', 'Earnings Growth Rate',
       'P/E Ratio Annual', 'P/E Ratio Quarter', 'Mean Recommendation',
       'Median Recommendation', 'Recommendation StdDev',
       'Number of Recommendations', 'Number of Upgrades',
       'Number of Downgrades', 'Buy Percentage', 'Sell Percentage',
       'Hold Percentage'],
      dtype='object')


In [23]:
# SICCD to Industry Mapping (Short Names)
siccd_mapping = {
    range(100, 1000): "Agriculture",
    range(1000, 1500): "Mining",
    range(1500, 1800): "Construction",
    range(2000, 4000): "Manufacturing",
    range(4000, 5000): "Transport",
    range(5000, 6000): "Retail",
    range(6000, 6800): "Finance",
    range(7000, 9000): "Services",
    range(9100, 10000): "Public",
}

detailed_siccd_mapping = {
    range(100, 1000): "Agriculture, Forestry, and Fishing",
    range(1000, 1500): "Mining and Quarrying",
    range(1500, 1800): "Construction",
    range(2000, 2400): "Food and Tobacco Manufacturing",
    range(2400, 2700): "Paper and Printing Manufacturing",
    range(2700, 2800): "Chemical and Pharmaceutical Manufacturing",
    range(2800, 2900): "Plastics and Rubber Products Manufacturing",
    range(3000, 3400): "Metals and Machinery Manufacturing",
    range(3400, 3600): "Electronics Manufacturing",
    range(3600, 3800): "Technology and Semiconductor Manufacturing",
    range(3800, 4000): "Miscellaneous Manufacturing",
    range(4000, 4500): "Transportation and Logistics",
    range(4500, 5000): "Communication Services and Utilities",
    range(5000, 5200): "Wholesale Trade",
    range(5200, 5600): "Retail Trade",
    range(5600, 6000): "Consumer Services",
    range(6000, 6200): "Banks and Credit Institutions",
    range(6200, 6400): "Insurance Companies",
    range(6400, 6800): "Real Estate and Investment Services",
    range(7000, 7300): "Hotels and Entertainment Services",
    range(7300, 7900): "Professional and Business Services",
    range(7900, 8000): "Healthcare and Social Services",
    range(8000, 8900): "Educational and Research Services",
    range(8900, 9000): "Other Services",
    range(9100, 9700): "Public Administration and Government",
    range(9700, 10000): "International Organizations"
}

def map_sic_to_industry(sic_code):
    for key, value in detailed_siccd_mapping.items():
        if sic_code in key:
            return value
    return "Others"
data['Real Industry'] = data['Industry Code'].apply(map_sic_to_industry)

In [12]:
selected_features = [
    'Weighted Mkt Return',
    'Price Growth Rate',
    'Price Ret Ex-Dividend',
    'Revenue Growth',
    'Earnings Growth Rate',
    'P/E Ratio Quarter',
    'Quart NI',
    'Mkt Cap',
    'Mean Recommendation',
    'EPS'
]

In [27]:
results_by_industry = {}

for industry, group in data.groupby('Real Industry'):
    # Prepare the data for regression
    #group['Next Quarter Excess Ret'] = group.groupby('TICKER')['Excess Return'].shift(-1)
    X = group[selected_features]
    #y = group['Next Quarter Excess Ret']
    y = group['Excess Return']

    # Add a constant term
    X = sm.add_constant(X)

    # Clean the data by removing inf, -inf, and NaN values
    X = X.replace([np.inf, -np.inf], np.nan).dropna()
    y = y[X.index]

    # Check for remaining missing values in the target variable
    if y.isna().sum() > 0:
        valid_indices = y.dropna().index
        X = X.loc[valid_indices]
        y = y.loc[valid_indices]

    # Standardize features for numerical stability
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

    # Fit the regression model
    model = sm.OLS(y, X_scaled).fit()

    # Save the summary of the regression results for the industry
    results_by_industry[industry] = model.summary()

# Display results for each industry
for industry, summary in results_by_industry.items():
    print(f"Results for {industry}:\n")
    print(summary)
    print("\n" + "="*80 + "\n")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regr

Results for Banks and Credit Institutions:

                                 OLS Regression Results                                
Dep. Variable:          Excess Return   R-squared (uncentered):                   0.962
Model:                            OLS   Adj. R-squared (uncentered):              0.961
Method:                 Least Squares   F-statistic:                              1527.
Date:                Tue, 03 Dec 2024   Prob (F-statistic):                        0.00
Time:                        12:20:52   Log-Likelihood:                          1307.4
No. Observations:                 618   AIC:                                     -2595.
Df Residuals:                     608   BIC:                                     -2551.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                            coef    std err          t      P>|t|      [0.02

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [26]:
#data['Next Quarter Excess Ret'] = data.groupby('TICKER')['Excess Return'].shift(-1)

X = data[selected_features]
#y = data['Next Quarter Excess Ret']
y = data['Excess Return']
X = sm.add_constant(X)

X = X.replace([np.inf, -np.inf], np.nan).dropna()
y = y[X.index]

# Check for remaining missing values in the target variable and clean if necessary
if y.isna().sum() > 0:
    valid_indices = y.dropna().index
    X = X.loc[valid_indices]
    y = y.loc[valid_indices]

# Standardize features for numerical stability
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

model = sm.OLS(y, X_scaled).fit()

print(model.summary())


                                 OLS Regression Results                                
Dep. Variable:          Excess Return   R-squared (uncentered):                   0.920
Model:                            OLS   Adj. R-squared (uncentered):              0.920
Method:                 Least Squares   F-statistic:                          2.055e+04
Date:                Tue, 03 Dec 2024   Prob (F-statistic):                        0.00
Time:                        12:19:59   Log-Likelihood:                          27752.
No. Observations:               17909   AIC:                                 -5.548e+04
Df Residuals:                   17899   BIC:                                 -5.541e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
